### Пайплайн для стекинга моделей

In [ ]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '32'
os.environ['NUMEXPR_NUM_THREADS'] = '30'
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [ ]:
import pandas as pd
from sklearn import metrics
import numpy as np

In [ ]:
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from lifelines.utils import concordance_index

import random
import warnings

warnings.simplefilter('ignore')

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

random.seed(42)
np.random.seed(42)

In [ ]:
main = pd.read_csv('data/train.csv')
real = pd.read_csv('data/sample_submit_naive.csv').drop('predict', axis=1)
main = pd.concat([main, real])

In [ ]:
test_ids = pd.read_csv('data/test_ids.csv')
train_, test_ = main.loc[~(main['user_id'].isin(test_ids['user_id'])) & ~(main['user_id'].isin(real['user_id']))], main.loc[main['user_id'].isin(test_ids['user_id'])]
train_ = train_.drop(['target', 'time'], axis=1)
test_ = test_.drop(['target', 'time'], axis=1)

In [ ]:
agg = pd.read_csv('predictions/agg_kdimon15_pred_meta.csv')
agg_1 = pd.read_csv('predictions/agg_0010_meta.csv')

emb = pd.read_csv('predictions/coles_coles-wtte-preds.csv')
emb_1 = pd.read_csv('predictions/coles_coles-wtte_clusters-preds.csv')
emb_2 = pd.read_csv('predictions/coles_wtte-rnn-preds.csv')

time = pd.read_csv('predictions/time_0010_pred_meta.csv')
time_1 = pd.read_csv('predictions/time_pred_meta.csv')

tfidf = pd.read_csv('predictions/agg_trx_td-idf.csv')

supervised = pd.read_csv('predictions/sup-preds.csv')
supervised_wtte = pd.read_csv('predictions/sup-wtte-preds.csv')

In [ ]:
main = main.merge(agg, on='user_id')
main = main.merge(agg_1, on='user_id')

main = main.merge(emb, on='user_id')
main = main.merge(emb_1, on='user_id')
main = main.merge(emb_2, on='user_id')


main = main.merge(time, on='user_id')
main = main.merge(time_1, on='user_id')

main = main.merge(tfidf, on='user_id')

main = main.merge(supervised, on='user_id')
main = main.drop(['index'], axis=1)
main = main.merge(supervised_wtte, on='user_id')
main['sp_x'] =  (main['sp_x'] - main['sp_x'].min()) / (main['sp_x'].max() - main['sp_x'].min())
main['sp_y'] =  (main['sp_y'] - main['sp_y'].min()) / (main['sp_y'].max() - main['sp_y'].min())

main

In [ ]:
train = train_.merge(main, on='user_id')
test = test_.merge(main, on='user_id')

In [ ]:
train

In [ ]:
X_train, y_train = train.drop(['target', 'user_id', 'time'], axis=1), train['target']
X_train

In [ ]:
from sklearn import linear_model
reg = linear_model.Ridge(alpha=12.85,
                         random_state = 42,
                        )
reg.fit(X_train, y_train)

In [ ]:
y_pred = reg.predict(test.drop(['target', 'user_id', 'time'], axis=1))
y_pred.shape

In [ ]:
reg.coef_

In [ ]:
print(metrics.roc_auc_score(test['target'], y_pred))

In [ ]:
concordance_index(test['time'], -y_pred, test['target'])

In [ ]:
real = real.merge(main, on='user_id')

In [ ]:
real

In [ ]:
y_pred = reg.predict(real.drop(['user_id', 'target', 'time'], axis=1))
y_pred.shape

In [ ]:
# Финальный предикт
pred = {'user_id': real.user_id.values, 'predict': y_pred}

pd.DataFrame(pred).to_csv('predictions/submission.csv', index=False)